### 1. Voting

In [227]:
using JuMP, Cbc
#data
repub = [80,60,40,20,40,40,70,50,70,70]
democ = [34,44,44,24,114,64,14,44,54,64]


m = Model(solver=CbcSolver())
@variable(m,x[1:5,1:10],Bin)
# number of democratic districts
@variable(m,y[1:5],Bin)
for j in 1:10
    @constraint(m,sum(x[i,j] for i in 1:5) == 1)
end
for i in 1:5
    @constraint(m,150 <= sum((repub[j] + democ[j])*x[i,j] for j in 1:10) <= 250)
end
for i in 1:5
    @constraint(m, sum((repub[j] - democ[j])*x[i,j] for j in 1:10) <= 200*(1-y[i]))
end
@objective(m,Max,sum(y[i] for i in 1:5))
status = solve(m)
println(status)
println("The number of congressional districts that have a Democratic majority is ",
    getobjectivevalue(m))

x = getvalue(x)

for i in 1:5
    list = Int64[]
    for j in 1:10
        if x[i,j] > 0
            append!(list,j)
        end
    end
    println("district ", i, ": cities", list)
end
;

Optimal
The number of congressional districts that have a Democratic majority is 3.0
district 1: cities[6, 9]
district 2: cities[1, 10]
district 3: cities[2, 7]
district 4: cities[5]
district 5: cities[3, 4, 8]


### 2. The Queens problem

In [228]:
# helper function to print a board grid
function printboard(arr)
    u = 0
    println(" ------------------------------- ")
    for p in 1:8
        print("| ")
        for r in 1:8
            u = round(Int, arr[p,r])
            u == 0 ? print(" ") : print(u)
            print(" | ")
        end
        println()
        println(" ------------------------------- ")
    end
end
;

In [229]:
#a

m = Model(solver=CbcSolver())
@variable(m,x[1:8,1:8],Bin)
#row
for j in 1:8
    @constraint(m,sum(x[i,j] for i in 1:8) == 1)
end
#column
for i in 1:8
    @constraint(m,sum(x[i,j] for j in 1:8) == 1)
end
#topleft
for i in 1:8
    @constraint(m,sum(x[i-j+1,j] for j in 1:i) <= 1)
end
#botleft
for i in 8:-1:1
    @constraint(m,sum(x[i-1+j,j] for j in 1:(8-i+1)) <= 1)
end
#botright
for i in 8:-1:1
    @constraint(m,sum(x[i-1+j,8-j+1] for j in 1:(8-i+1)) <= 1)
end
#topright
for i in 1:8
    @constraint(m,sum(x[i-j+1,8-j+1] for j in 1:i) <= 1)
end
@objective(m,Max,sum(x[i,j] for i in 1:8, j in 1:8))

status = solve(m)
println(status)
x2 = getvalue(x)
printboard(x2)
;

Optimal
 ------------------------------- 
|   |   |   | 1 |   |   |   |   | 
 ------------------------------- 
|   |   |   |   |   |   |   | 1 | 
 ------------------------------- 
| 1 |   |   |   |   |   |   |   | 
 ------------------------------- 
|   |   | 1 |   |   |   |   |   | 
 ------------------------------- 
|   |   |   |   |   | 1 |   |   | 
 ------------------------------- 
|   | 1 |   |   |   |   |   |   | 
 ------------------------------- 
|   |   |   |   |   |   | 1 |   | 
 ------------------------------- 
|   |   |   |   | 1 |   |   |   | 
 ------------------------------- 


In [230]:
#b

m = Model(solver=CbcSolver())
@variable(m,x[1:8,1:8],Bin)
#row
for j in 1:8
    @constraint(m,sum(x[i,j] for i in 1:8) == 1)
end
#column
for i in 1:8
    @constraint(m,sum(x[i,j] for j in 1:8) == 1)
end
#topleft
for i in 1:8
    @constraint(m,sum(x[i-j+1,j] for j in 1:i) <= 1)
end
#botleft
for i in 8:-1:1
    @constraint(m,sum(x[i-1+j,j] for j in 1:(8-i+1)) <= 1)
end
#botright
for i in 8:-1:1
    @constraint(m,sum(x[i-1+j,8-j+1] for j in 1:(8-i+1)) <= 1)
end
#topright
for i in 1:8
    @constraint(m,sum(x[i-j+1,8-j+1] for j in 1:i) <= 1)
end
#symmetry
for i in 1:8
    for j in 1:8
        @constraint(m,x[i,j] == x[8-i+1,8-j+1])
    end
end

@objective(m,Max,sum(x[i,j] for i in 1:8, j in 1:8))

status = solve(m)
println(status)
x2 = getvalue(x)
printboard(x2)
;

Optimal
 ------------------------------- 
|   |   |   |   |   | 1 |   |   | 
 ------------------------------- 
|   |   |   | 1 |   |   |   |   | 
 ------------------------------- 
|   |   |   |   |   |   | 1 |   | 
 ------------------------------- 
| 1 |   |   |   |   |   |   |   | 
 ------------------------------- 
|   |   |   |   |   |   |   | 1 | 
 ------------------------------- 
|   | 1 |   |   |   |   |   |   | 
 ------------------------------- 
|   |   |   |   | 1 |   |   |   | 
 ------------------------------- 
|   |   | 1 |   |   |   |   |   | 
 ------------------------------- 


In [231]:
#helper function to return counts on diagonals

#check diagonal with slope -1
function checkdiag1(i,j,arr)
    count = 0
    while i > 1 && j > 1
        i -= 1
        j -= 1
    end
    while i <= 8 && j <= 8
        count += arr[i,j]
        i += 1
        j += 1
    end
    return count
end

#check diagonal with slope 1
function checkdiag2(i,j,arr)
    count = 0
    while i < 8 && j > 1
        i += 1
        j -= 1
    end
    while i >= 8 && j <= 8
        count += arr[i,j]
        i -= 1
        j += 1
    end
    return count
end
;

In [232]:
#c

m = Model(solver=CbcSolver())
@variable(m,x[1:8,1:8],Bin)

for i in 1:8
    for j in 1:8
    @constraint(m, checkdiag1(i,j,x) + checkdiag2(i,j,x) + 
            sum(x[i,j] for j in 1:8) + sum(x[i,j] for i in 1:8) >= 1)
    end
end
@objective(m,Min,sum(x[i,j] for i in 1:8, j in 1:8))

status = solve(m)
println(status)
x2 = getvalue(x)
printboard(x2)
println("At least need ", getobjectivevalue(m), " queens")
;

Optimal
 ------------------------------- 
|   |   |   |   |   |   |   |   | 
 ------------------------------- 
|   |   |   |   |   | 1 |   |   | 
 ------------------------------- 
|   | 1 |   |   |   |   |   |   | 
 ------------------------------- 
|   |   |   |   |   |   |   |   | 
 ------------------------------- 
|   |   |   |   |   |   | 1 |   | 
 ------------------------------- 
|   |   |   |   |   |   |   |   | 
 ------------------------------- 
|   |   |   | 1 |   |   |   |   | 
 ------------------------------- 
| 1 |   |   |   |   |   |   |   | 
 ------------------------------- 
At least need 5.0 queens


In [233]:
#d

m = Model(solver=CbcSolver())
@variable(m,x[1:8,1:8],Bin)

for i in 1:8
    for j in 1:8
    @constraint(m, checkdiag1(i,j,x) + checkdiag2(i,j,x) + 
            sum(x[i,j] for j in 1:8) + sum(x[i,j] for i in 1:8) >= 1)
    end
end

#symmetry
for i in 1:8
    for j in 1:8
        @constraint(m,x[i,j] == x[8-i+1,8-j+1])
    end
end
@objective(m,Min,sum(x[i,j] for i in 1:8, j in 1:8))

status = solve(m)
println(status)
x2 = getvalue(x)
printboard(x2)
println("At least need ", getobjectivevalue(m), " queens")
;

Optimal
 ------------------------------- 
|   | 1 |   |   |   |   |   |   | 
 ------------------------------- 
|   |   |   |   |   |   |   |   | 
 ------------------------------- 
|   |   |   | 1 |   |   |   |   | 
 ------------------------------- 
|   |   |   |   |   |   |   | 1 | 
 ------------------------------- 
| 1 |   |   |   |   |   |   |   | 
 ------------------------------- 
|   |   |   |   | 1 |   |   |   | 
 ------------------------------- 
|   |   |   |   |   |   |   |   | 
 ------------------------------- 
|   |   |   |   |   |   | 1 |   | 
 ------------------------------- 
At least need 6.0 queens


### 3. Relay Race


In [234]:
#data
toTime = [0 1.1 1.3 1.9 2.1
 1.2 0 1.7 1.0 1.8
 1.7 1.4 0 0.9 1.7
 2.1 0.8 1.6 0 2.4
 1.5 1.2 1.9 2.3 0]
name = ["Alice" "Bob" "Cindy" "David" "Elisa"]
;

In [235]:
m = Model(solver=CbcSolver())
@variable(m,x[1:5,1:5],Bin)

#each person has only one person to pass on the baton
for j in 1:5
    @constraint(m,sum(x[i,j] for i in 1:5) <= 1)
end
for i in 1:5
    #cannot pass to themselves
    @constraint(m,x[i,i] == 0)
    #each person gets one chance
    @constraint(m,sum(x[i,j] for j in 1:5) <= 1)
end
#total 4 pass-ons
@constraint(m,sum(x[i,j] for i in 1:5, j in 1:5) == 4)
#everyone should participate in this race
for i in 1:5
    @constraint(m,sum(x[i,j] for j in 1:5) + sum(x[j,i] for j in 1:5) >= 1)
end

@objective(m,Min, sum(x[i,j]*toTime[i,j] for i in 1:5, j in 1:5))
status = solve(m)
println(status)
x = getvalue(x)
start = 0
for i in 1:5
    if(sum(x[i,j] for j in 1:5) == 0)
        start = i
        break
    end
end
print("Path: ",name[start])
curr = start
for k in 1:4
    for i in 1:5
        if(x[i,curr] == 1)
            curr = i
            print("->", name[curr])
            break
        end
    end
end
println()
println("total time: ", 82.5+77.1+81.3+74.9+80.6 + getobjectivevalue(m),"s")
;

Optimal
Path: Bob->David->Cindy->Alice->Elisa
total time: 400.9s
